In [1]:
#  install several Python packages,
!pip -q install langchain openai tiktoken chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 29.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 70.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.6/123.6 kB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 72.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.6/62.6 kB 7.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 965.1/965.1 kB 70.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.0/57.0 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 45.1 MB/

In [5]:
!pip install unzip

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for unzip: filename=unzip-1.0.0-py3-none-any.whl size=1279 sha256=4a8a6e46e8b4ba6d6af5cbc9cda9b6d45da09257e8904077f6ede57e493a9b01
  Stored in directory: /root/.cache/pip/wheels/80/dc/7a/f8af45bc239e7933509183f038ea8d46f3610aab82b35369f4
Successfully built unzip


# LangChain multi-doc retriever with ChromaDB

# Setting up LangChain

In [16]:
import os

os.environ["OPENAI_API_KEY"] = "your key"

In [13]:
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.llms import OpenAI
from langchain.chains import RetrievalQA
from langchain.document_loaders import TextLoader
from langchain.document_loaders import DirectoryLoader

# Load multiple and process documents

In [14]:
# data source, https://themarketswatch.com/financial/top-stocks-to-buy/?gclid=CjwKCAjwyqWkBhBMEiwAp2yUFpSLJZSVwVHJR3co5yVHVLeBPqNNcU3VMrFJjniP3jtuupP6kHzDYxoC8dkQAvD_BwE
import zipfile
!wget https://github.com/Shafi2016/Youtube/raw/main/stock_market_june_2023.zip -O stock_market_june_2023.zip  # Downloads the zip file from the specified URL.

with zipfile.ZipFile('stock_market_june_2023.zip', 'r') as zip_ref:
    zip_ref.extractall()

--2023-06-17 16:18:58--  https://github.com/Shafi2016/Youtube/raw/main/stock_market_june_2023.zip
Resolving github.com (github.com)... 140.82.121.4
Connecting to github.com (github.com)|140.82.121.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/Shafi2016/Youtube/main/stock_market_june_2023.zip [following]
--2023-06-17 16:18:59--  https://raw.githubusercontent.com/Shafi2016/Youtube/main/stock_market_june_2023.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10638 (10K) [application/zip]
Saving to: ‘stock_market_june_2023.zip’

stock_market_june_2 100%[===================>]  10.39K  --.-KB/s    in 0s      

2023-06-17 16:18:59 (112 MB/s) - ‘stock_market_june_2023.zip’ saved [10638/10638]



In [15]:

# Initializes a DirectoryLoader to load text files from the specified directory.
loader = DirectoryLoader('./stock_market_june_2023/', glob="./*.txt", loader_cls=TextLoader)

# Loads all the documents present in the directory.
documents = loader.load()




In [10]:

# Initializes a RecursiveCharacterTextSplitter with a specified chunk size and overlap.
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)

# Splits the loaded documents into chunks of text using the defined text_splitter.
texts = text_splitter.split_documents(documents)


In [11]:
len(texts)

49

In [12]:
texts[0]

Document(page_content='The global shift towards electric vehicles (EVs) is driving an exponential increase in electricity demand. Countries worldwide, including oil-producing nations, are turning to nuclear energy to meet this growing need while transitioning away from fossil fuels. Consequently, the demand for uranium, essential for fueling the nuclear power sector, has experienced a significant surge in the past year after decades of stagnation, clearly showcased in the uranium spot price.', metadata={'source': 'stock_market_june_2023/Kiplin Metals Inc.txt'})

# create the DB

In [17]:
# Sets the directory where the embeddings will be stored.
persist_directory = 'db'

# Sets the OpenAI Embeddings object as the embedding to use.
embedding = OpenAIEmbeddings()

# Uses the Chroma module to convert the texts into embeddings using the  OpenAI Embeddings
# The resulting embeddings are stored in the persist_directory.
vectordb = Chroma.from_documents(documents=texts,
                                 embedding=embedding,
                                 persist_directory=persist_directory)


In [18]:

# Persists the generated embeddings database to disk.
vectordb.persist()

# Releases the memory held by the vectordb object by setting it to None.
vectordb = None


In [19]:

# Re-initializes the Chroma object from the persisted directory with the specified embeddings.
vectordb = Chroma(persist_directory=persist_directory, embedding_function=embedding)


# Make a retriever

In [20]:
# Initializes a retriever object from the vectordb with a specified number of search results (k=3).
# The vectordb.as_retriever() function is likely converting the Chroma object, which contains vectorized representations of text, into a retriever object.
# The retriever can be used to find the most similar vectors in the database given a query vector.
# The search_kwargs={"k": 3} argument suggests that the retriever will return the top 3 most similar vectors for each query.
retriever = vectordb.as_retriever(search_kwargs={"k": 3})


# Make a chain

In [21]:
# Initializes a RetrievalQA object with the specified OpenAI(), retriever, and chain type.
# The return_source_documents parameter set to True means the original source documents will be included in the returned results.
qa_chain = RetrievalQA.from_chain_type(llm=OpenAI(),
                                  chain_type="stuff",
                                  retriever=retriever,
                                  return_source_documents=True)


In [22]:

 # This function processes the response from a Language Model (llm) and prints the result and sources.
def process_llm_response(llm_response):
    # Prints the 'result' field from the response, which likely contains the answer from the language model.
    print(llm_response['result'])

    print('\n\nSources:')
    # Iterates over the 'source_documents' field in the response, which contains the documents where the answer was found.
    for source in llm_response["source_documents"]:
        # Prints the 'source' metadata from each document.
        print(source.metadata['source'])



In [23]:
query = "Could you please enumerate the companies that have been highlighted for their potential stock growth"
llm_response = qa_chain(query)
process_llm_response(llm_response)

 The companies that have been highlighted for their potential stock growth are Arbor Metals Corp., Tesla Inc., Kiplin Metals Inc., Microsoft, NVIDIA, Shopify, Intuitive Surgical, and MercadoLibre.


Sources:
stock_market_june_2023/stock_market.txt
stock_market_june_2023/stock_market.txt
stock_market_june_2023/stock_market.txt


In [28]:
query = "how many companies that have been highlighted for their potential stock growth"
llm_response = qa_chain(query)
process_llm_response(llm_response)

 8


Sources:
stock_market_june_2023/stock_market.txt
stock_market_june_2023/stock_market.txt
stock_market_june_2023/stock_market.txt


In [24]:
query = "What are the top five companies and how much did their stock value increase?"
llm_response = qa_chain(query)
process_llm_response(llm_response)


 Palantir soared by over 74%, Nvidia witnessed a growth of over 34%, Crowdstrike Holdings increased by over 30%, Shopify saw a rise of over 19%, and Our insights and articles about these stocks collectively yielded a remarkable 57% return last month for our readers who took action on our recommendations.


Sources:
stock_market_june_2023/stock_market.txt
stock_market_june_2023/stock_market.txt
stock_market_june_2023/Arbor Metals Corp.txt


In [25]:
query = "How much has Microsoft invested in OpenAI?"
llm_response = qa_chain(query)
process_llm_response(llm_response)


 Microsoft has invested $1 billion in OpenAI.


Sources:
stock_market_june_2023/Microsoft Corporation.txt
stock_market_june_2023/Microsoft Corporation.txt
stock_market_june_2023/Microsoft Corporation.txt


In [26]:
query = "What were the reasons behind Shopify's decision to lay off a portion of its workforce"
llm_response = qa_chain(query)
process_llm_response(llm_response)

 To reduce overhead costs and make Shopify more asset-light in the long run.


Sources:
stock_market_june_2023/Shopify.txt
stock_market_june_2023/Shopify.txt
stock_market_june_2023/Shopify.txt


In [27]:
query = "How does NVIDIA's dominance in the GPU market contribute to its strategic position in the AI industry?"
llm_response = qa_chain(query)
process_llm_response(llm_response)

 NVIDIA's dominance in the GPU market allows them to provide hardware, like their A100 and H100 chips, to AI platforms like ChatGPT and Alphabet's A3 supercomputers. This reinforces confidence in NVIDIA's technological leadership and has helped drive their high share prices.


Sources:
stock_market_june_2023/NVIDIA Corporation.txt
stock_market_june_2023/NVIDIA Corporation.txt
stock_market_june_2023/NVIDIA Corporation.txt
